In [1]:
import pandas as pd
dataset = pd.read_csv(r"C:\Users\devin\Downloads\hate_speech.csv")
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [2]:
dataset.shape

(5242, 3)

In [5]:
dataset.label.value_counts()

label
0    3000
1    2242
Name: count, dtype: int64

In [7]:
for index, tweet in enumerate(dataset["tweet"][10:15]):
  print(index+1, "-", tweet)

1 -  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 - we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 - i get to see my daddy today!!   #80days #gettingfed
4 - ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 - i am thankful for having a paner. #thankful #positive     


In [9]:
import re
def clean_text(text):
  text = re.sub(r'[^a-zA-Z]',' ', text)
  text = re.sub(r'[^\x00-\x7F]+', ' ', text)
  text = text.lower()
  return text

In [11]:
dataset['clean text'] = dataset.tweet.apply(lambda x: clean_text(x))

In [13]:
dataset.head(10)

,id,label,tweet,clean text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can t us...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide society now motivation
5,6,0,[2/2] huge fan fare and big talking before the...,huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...,user camping tomorrow user user user use...
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams ...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...,user user welcome here i m it s so gr...


In [15]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
len(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\devin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


179

In [16]:
stop = stopwords.words('english')

In [17]:
def gen_freq(text):
  word_list = []
  for tw_words in text.split():
    word_list.extend(tw_words)
  word_freq = pd.Series(word_list).value_counts()
  word_freq = word_freq.drop(stop, errors='ignore')
  return word_freq

In [18]:
def any_neg(words):
  for word in words:
    if word in ['no', 'nor','non', 'not'] or re.search(r"\wn't", word):
      return 1
  else:
    return 0

In [19]:
def any_rare(words, rare_100):
  for word in words:
    if word in rare_100:
      return 1
    else:
      return 0

In [20]:
def is_question(words):
  for word in words:
    if word in ['when', 'what', 'how', 'why', 'who', 'where']:
      return 1
    else:
      return 0

In [27]:
dataset.head(10)

,id,label,tweet,clean text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can t us...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide society now motivation
5,6,0,[2/2] huge fan fare and big talking before the...,huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...,user camping tomorrow user user user use...
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams ...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...,user user welcome here i m it s so gr...


In [29]:
from sklearn.model_selection import train_test_split

In [31]:
dataset['clean_text'] = dataset['tweet'].apply(lambda x: clean_text(x))

dataset['word_count'] = dataset['clean_text'].apply(lambda x: len(x.split()))
dataset['char_count'] = dataset['clean_text'].apply(lambda x: len(x))
dataset['any_neg'] = dataset['clean_text'].apply(lambda x: any_neg(x.split()))
dataset['is_question'] = dataset['clean_text'].apply(lambda x: is_question(x.split()))

rare_100 = gen_freq(' '.join(dataset['clean_text'])).tail(100).index.tolist()
dataset['any_rare'] = dataset['clean_text'].apply(lambda x: any_rare(x.split(), rare_100))

X = dataset[['word_count', 'any_neg', 'any_rare', 'char_count', 'is_question']]
y = dataset['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("Train and test data split successful!")

Train and test data split successful!


In [33]:
word_freq = gen_freq(dataset.clean_text.str)
rare_100 = word_freq[-100:]
dataset['word_count']=dataset.clean_text.str.split().apply(lambda x:len(x))
dataset['any_neg']=dataset.clean_text.str.split().apply(lambda x:any_neg(x))
dataset['is_question']=dataset.clean_text.str.split().apply(lambda x:is_question(x))
dataset['any_rare']=dataset.clean_text.str.split().apply(lambda x:any_rare(x,rare_100))
dataset['char_count']=dataset.clean_text.apply(lambda x:len(x))

In [ ]:
from sklearn.model_selection import train_split

X = dataset[['word_count', 'any_neg', 'any_race','char_count', 'is_question']]
y = dataset.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

In [37]:
from sklearn.naive_bayes import GaussianNB

model= GaussianNB()
model = model.fit(X_train,y_train)
pred= model.predict(X_test)

In [39]:
model.predict(X_test[5:10])

array([0, 0, 1, 1, 0], dtype=int64)

In [41]:
from sklearn.metrics import accuracy_score

print("Accuracy: ", accuracy_score(y_test,pred)*100,"%")

Accuracy:  54.33746425166825 %


In [43]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier()
clf_rf.fit(X_train, y_train)
rf_pred = clf_rf.predict(X_test).astype(int)

In [45]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,rf_pred))
print(classification_report (y_test,rf_pred))
print("Accuracy:", accuracy_score(y_test,rf_pred))

[[414 185]
 [232 218]]
              precision    recall  f1-score   support

           0       0.64      0.69      0.67       599
           1       0.54      0.48      0.51       450

    accuracy                           0.60      1049
   macro avg       0.59      0.59      0.59      1049
weighted avg       0.60      0.60      0.60      1049

Accuracy: 0.6024785510009533


In [ ]:
from sklearn.linear_model import Logisticregression
logreg = LogisticRegression(class_weight='balanced')
logreg.fit(X_train,y_train)

In [ ]:
y_pred=logreg.predict(X_test